### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
#type(nyt_api_key)
type(tmdb_api_key)

### Access the New York Times API

In [ ]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
pagect = str(0)
query_url = url + "api-key=" + nyt_api_key + "&fq=" + filter_query + "&sort=" + sort + "&begin_date=" + begin_date \
                + "&end_date=" + end_date + "&fl=" + field_list 
query_url

In [ ]:
# Create an empty list to store the reviews

articles_list = []
pagect = 0
article_count = 0

# loop through pages 0-19

    # create query with a page number
    # API results show 10 articles at a time

    
    # Make a "GET" request and retrieve the JSON

    
    # Add a twelve second interval between queries to stay within API query limits

    
    # Try and save the reviews to the reviews_list

        # loop through the reviews["response"]["docs"] and append each review to the list

        # Print the page that was just retrieved


        # Print the page number that had no results then break from the loop
articles  = requests.get(query_url).json()
print (f' Page {pagect} retrieved')
while pagect < 19 and articles["response"]["docs"] != []:
    for article in articles["response"]["docs"]:
        articles_list.append(article) 
        article_count += 1   
    pagect += 1
    time.sleep(12)
    query_url_page = f"{query_url}&page={str(pagect)}"
#    print (f' {query_url_page}')
    articles  = requests.get(query_url_page).json()
    print (f' Page {pagect} retrieved with {article_count} articles stored.')
if articles["response"]["docs"] == []:
    print (f'This page number -{pagect}- did not have any results. ')
else:
    print (f'Saving final group of articles on final page...')
    for article in articles["response"]["docs"]:
        articles_list.append(article) 
        article_count += 1 
print (f'Operation complete.  Total of {article_count} articles stored.')        
    



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(articles_list, indent=4))


In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
articles_normalized_df = pd.json_normalize(articles_list)
articles_normalized_df


In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
articles_normalized_df['title'] = articles_normalized_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
articles_normalized_df


In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords


# Fix the "keywords" column by converting cells from a list to a string

articles_normalized_df['keywords'] = articles_normalized_df['keywords'].apply(lambda x: extract_keywords(x))
articles_normalized_df



In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = []
title_list = articles_normalized_df["title"].to_list()
title_list

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
movie_title = 'The Attachment Diaries'

query_url = url + movie_title + tmdb_key_string
query_url

urlmid = "https://api.themoviedb.org/3/movie/"
tmdb_key_string2 ="?api_key=" + tmdb_api_key
movie_id = '743040'

query_url_mid = urlmid + movie_id + tmdb_key_string2

#query_url_mid
query_url


In [ ]:
# Create an empty list to store the results

tmdb_movies_list = []
dicofflicks = []


# Create a request counter to sleep the requests after a multiple
# of 50 requests

request_counter = 1

# Loop through the titles
for title in title_list:
    movie_title = title
    query_url = url + movie_title + tmdb_key_string
    listofgenres = []
    listoflang = []
    countrylist = []
    # Perform a "GET" request for The Movie Database

       # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

    try:
        movie  = requests.get(query_url).json()
        if movie["results"] != []:        
            tmdb_movies_list.append(movie)
        # Print out the title that was found
            print(f'This movie title was found...{movie_title}')
        # We have a valid movie title here.  Let's get the details....
        # Get movie id
            movie_id = str(movie['results'][0]['id'])
        #    print(f'Extracted movie id = {movie_id}')

        # Make a request for a the full movie details

            query_url_mid = urlmid + movie_id + tmdb_key_string2
        
        # Execute "GET" request with url
            try:
                movie  = requests.get(query_url_mid).json()  # get full movie details
                if movie != []:
            #    tmdb_full_movies_list.append(movie)  # save away for now
            #        print(f'Full movie details obtained for {movie_id}')

                # Extract the genre names into a list        
                    for x in range(len(movie['genres'])):
                        listofgenres.append(movie['genres'][x]['name'])
            #        print(f'{listofgenres}')
            #       
                # Extract the spoken_languages' English name into a list    
                    for x in range(len(movie['spoken_languages'])):
                        listoflang.append(movie['spoken_languages'][x]['english_name'])
            #        print(f'{listoflang}')
            #
                # Extract the production_countries' name into a list
                    for x in range(len(movie['production_countries'])):
                        countrylist.append(movie['production_countries'][x]['name'])
            #        print(f'{countrylist}')
            #
                # Add the relevant data to a dictionary
                    dicofflicks.append({    
                                "title": movie_title,    
                                "original_title": movie['original_title'],
                                "homepage": movie['homepage'],
                                "overview": movie['overview'],
                                "popularity": movie['popularity'],
                                "runtime": movie['runtime'],
                                "revenue": movie['revenue'],
                                "release_date": movie['release_date'],
                                "vote_average": movie['vote_average'],
                                "vote_count": movie['vote_count'],
                                "genres":listofgenres,
                                "spoken_languages":listoflang,
                                "production_countries":countrylist
                                })
                #    print(f'{dicofflicks}')
                else:             
                    print(f' No Movie found for movie id {movie_id}')
            except: 
                if movie == []:
                    print(f' No Movie found for movie id {movie_id}')
                else:
                    print(f' Exception occurred.  Program attempting to continue.')
        else:
            print(f'Movie Title Not Found...{movie_title}')
    except:
        if movie["results"] == []:      
            print(f'Movie Title Not Found...{movie_title}')
        else:
            print(f' Exception occurred.  Program attempting to continue.')            
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print('')
        print (f' {request_counter} requests have completed, taking a breather.')
        print('')
        time.sleep(1)
        request_counter += 1
    else:
        request_counter += 1
        # Add 1 to the request counter 


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

tmdb_movies_list.append(dicofflicks)




In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(dicofflicks, indent=4))

In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(dicofflicks)


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
# merged_df = articles_normalized_df.merge(tmdb_df, on='title')
merged_df = tmdb_df.merge(articles_normalized_df, on='title')
merged_df.head()

In [25]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']


# Create a list of characters to remove


def weedout(list_entry):  # weedout the special chars --> }{' <--
    characters_to_remove = ['[', ']', "'"]
    work = str(list_entry)
    for y in characters_to_remove:
            result = work.replace(y,"")
            work = result
    return work
      
def coltype(col):      # function to convert column to string
        merged_df.astype({col: "string"})    

    # Convert the column to type 'str'

for col in columns_to_fix:   # Iterate through to convert columns to string
        coltype(col)

   # Loop through characters to remove

for col in columns_to_fix:
        merged_df[col] = merged_df[col].apply(lambda row: weedout(row))
        



# Display the fixed DataFrame

merged_df.head()


In [ ]:
#print (f'{merged_df[ "genres"]}')
#print (f'{merged_df["spoken_languages"]}')
#print (f'{merged_df["production_countries"]}')
merged_df.head()
#merged_df.columns





In [ ]:
# Drop "byline.person" column
cleaned_df = merged_df.drop("byline.person", axis=1)

In [ ]:
# Delete duplicate rows and reset index
cleaned_df.drop_duplicates
cleaned_df.reset_index()


In [27]:
# Export data to CSV without the index
from pathlib import Path  

filepath = Path('./MovieData.csv')  
  

cleaned_df.to_csv(filepath, index=False)  